## ETF Compete:  
## Feature engineering: RSI (相對強勢指標) 

https://www.cmoney.tw/notes/note-detail.aspx?nid=6162

## n 天內漲跌強弱相對指標
### Date: 18424_L

# 使用變數


KD 公式
$$$$
# av_up = (n days 上漲的總和)/n
# av_dn = (n days 下跌的總和)/n
# $RS = \frac{Av_{up}}{(Av_{up}+Av_{dn})}$
# RSI= $\frac{RS}{1+RS}$ or equivantly, RSI = $1 - \frac{1}{1+RS}$



### variable: 
### df: dataframe 
### n: n days window (normally, 7 (6天漲跌) )


###  return df 
### feature columns = [ 'RS', 'RSI']
### note： NaN value from Row 0 to 5

In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time

In [2]:
# read in csv, change column names
etf = pd.read_csv('/Users/LarryGuo/Desktop/nano_degree/Capstone_Talk/ETF_compete/twetf_utf8/tetfp.csv')
col_dtypes = OrderedDict(code=str, date=str, name=str, open=float, high=float, low=float, close=float, volume=int)

etf.columns=col_dtypes.keys()

etf.head()

,code,date,name,open,high,low,close,volume
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507


In [3]:

etf= etf[:1286] # select 台灣50 
etf.tail()

,code,date,name,open,high,low,close,volume
1281,50,20180327,元大台灣50,83.00,83.40,82.95,83.40,3277
1282,50,20180328,元大台灣50,82.90,82.90,82.20,82.25,4161
1283,50,20180329,元大台灣50,82.25,82.35,81.80,82.10,4099
1284,50,20180330,元大台灣50,82.65,83.05,82.65,82.85,4994
1285,50,20180331,元大台灣50,82.85,83.05,82.75,82.95,878


In [4]:
etf.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [7]:
# calculate KD_RSV

def calculate_up_dn(n_df):
    #print ('len_n_df=', len(n_df))
    up = n_df[n_df['diff']>=0]['diff'].sum()
    down = n_df[n_df['diff']< 0]['diff'].abs().sum()
    
    return up, down

def RSI(df,n=7):
    start_time=time()
    
    # create new column and assign initial value 
    # K_signal = 1 if K_new > K_old
    
    #df['av_up_old'] = 0.5
    df['av_up'] = 0
    #df['av_dn_old'] = 0.5
    df['av_dn'] = 0
    df['prev_close'] = df['close'].shift(1)
    df['diff'] = df['close'] - df['prev_close']

    
    for j in range(n-1,len(df)):
    #for j in range(6,30):
        df['av_up'].iloc[j] = calculate_up_dn(df[j-n+1:j+1])[0]
        df['av_dn'].iloc[j] = calculate_up_dn(df[j-n+1:j+1])[1]

  
    
    
    df['RS'] = df['av_up']/(df['av_up']+df['av_dn'])

    
    df['RSI'] = df['RS']/(1+df['RS'])
    

    
    
    
    return df
 
   

In [8]:
df= RSI(etf)

/Users/LarryGuo/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
df.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume',
       'av_up', 'av_dn', 'prev_close', 'diff', 'RS', 'RSI'],
      dtype='object')

In [10]:
df

,code,date,name,open,high,low,close,volume,av_up,av_dn,prev_close,diff,RS,RSI
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487,0.00,0.00,NaN,NaN,NaN,NaN
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020,0.00,0.00,54.40,0.45,NaN,NaN
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837,0.00,0.00,54.85,-0.35,NaN,NaN
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910,0.00,0.00,54.50,-0.25,NaN,NaN
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507,0.00,0.00,54.25,-0.35,NaN,NaN
5,50,20130109,元大台灣50,53.75,54.30,53.75,54.10,7529,0.00,0.00,53.90,0.20,NaN,NaN
6,50,20130110,元大台灣50,54.30,54.65,54.15,54.50,13953,1.05,0.95,54.10,0.40,0.525000,0.344262
7,50,20130111,元大台灣50,54.70,54.80,54.35,54.45,11837,1.05,1.00,54.50,-0.05,0.512195,0.338710
8,50,20130114,元大台灣50,54.00,54.50,53.80,54.50,7282,0.65,1.00,54.45,0.05,0.393939,0.282609
9,50,20130115,元大台灣50,54.20,54.45,53.90,54.00,6609,0.65,1.15,54.50,-0.50,0.361111,0.265306
